In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import os

seed = 2023

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
sbert_embeddings.shape

(5568, 384)

In [8]:
x_data = obtain_path(
    df=df_rumours, 
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=20
)

x_data.shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

(5568, 20, 384)

## Baseline: BiLSTM

In [11]:
num_epochs = 100
hidden_dim_sizes = [100, 200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

## history length = 20

In [12]:
size = 20
bilstm_history_20, best_bilstm_history_20, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2_best_model.csv


In [13]:
bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/1802967379.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.275409  0.545281  0.541669   
                        0.0005         0.423434  0.530982  0.521996   
                        0.0010         0.409979  0.523991  0.518827   
           0.5          0.0001         0.337892  0.533206  0.530322   
                        0.0005         0.456228  0.532253  0.529302   
                        0.0010         0.385305  0.515094  0.505986   
200        0.1          0.0001         0.533800  0.511916  0.507559   
                        0.0005         0.444130  0.526851  0.519304   
                        0.0010         0.410281  0.534477  0.533793   
           0.5          0.0001         0.527590  0.523356  0.521872   
                        0.0005         0.424295  0.537019  0.532046   
                        0.0010         0.418495  0.513187  0.508559   
300        0.1          0.0001         0.484102  0.530346  0.526533   
                        0.0005         0.404987  0.520813  0.516337   
                        0.0010         0.399257  0.542104  0.532823   
           0.5          0.0001         0.591023  0.531300  0.527002   
                        0.0005         0.477116  0.522402  0.515327   
                        0.0010         0.380512  0.513823  0.506332   
384        0.1          0.0001         0.439538  0.525898  0.521486   
                        0.0005         0.428475  0.508421  0.500730   
                        0.0010         0.379192  0.528440  0.518970   
           0.5          0.0001         0.510897  0.522085  0.518291   
                        0.0005         0.430627  0.515729  0.505524   
                        0.0010         0.401587  0.535113  0.527562   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.549905  0.547902    0.252523   
                        0.0005          0.530148  0.528171    0.339659   
                        0.0010          0.524409  0.523902    0.323415   
           0.5          0.0001          0.535560  0.534610    0.288194   
                        0.0005          0.534429  0.533521    0.387965   
                        0.0010          0.515897  0.514093    0.292899   
200        0.1          0.0001          0.511819  0.511265    0.351796   
                        0.0005          0.523209  0.523600    0.427234   
                        0.0010          0.537632  0.537104    0.351665   
           0.5          0.0001          0.526075  0.525705    0.346581   
                        0.0005          0.537800  0.536795    0.407201   
                        0.0010          0.513520  0.513402    0.346286   
300        0.1          0.0001          0.536748  0.534913    0.370129   
                        0.0005          0.520133  0.520450    0.348208   
                        0.0010          0.560008  0.547484    0.333889   
           0.5          0.0001          0.538044  0.535883    0.473150   
                        0.0005          0.520402  0.520986    0.431940   
                        0.0010          0.514832  0.514307    0.301376   
384        0.1          0.0001          0.533338  0.531347    0.349238   
                        0.0005          0.506754  0.506855    0.341458   
                        0.0010          0.539863  0.534608    0.272771   
           0.5          0.0001          0.527807  0.526067    0.343338   
                        0.0005          0.518521  0.517110    0.357760   
                        0.0010          0.546447  0.541936    0.322743   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.696323  0.648573   
                        0.0005               0.727165  0.680084   
   

In [14]:
best_bilstm_history_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.640989,0.533842,0.531989,"[0.5025432349949135, 0.5614349775784753]",0.539034,"[0.5691244239631337, 0.5089430894308943]",0.537954,"[0.44990892531876137, 0.626]",0.516310,0.747331,...,True,300,0.5,0.0001,1,focal,2,False,None,64
0,0.529864,0.530982,0.519953,"[0.44719101123595506, 0.5927152317880794]",0.544614,"[0.5835777126099707, 0.5056497175141242]",0.539239,"[0.36247723132969034, 0.716]",0.359668,0.722420,...,True,300,0.5,0.0001,12,focal,2,False,None,64
0,0.602217,0.529075,0.529065,"[0.5268199233716475, 0.5313092979127134]",0.530485,"[0.5555555555555556, 0.5054151624548736]",0.530455,"[0.5009107468123861, 0.56]",0.543472,0.736655,...,True,300,0.5,0.0001,123,focal,2,False,None,64


In [15]:
best_bilstm_history_20["f1"].mean()

0.5270022794802974

In [16]:
best_bilstm_history_20["precision"].mean()

0.5380442769214254

In [17]:
best_bilstm_history_20["recall"].mean()

0.535882817243473

In [18]:
np.stack(best_bilstm_history_20["f1_scores"]).mean(axis=0)

array([0.49218472, 0.56181984])

In [19]:
np.stack(best_bilstm_history_20["precision_scores"]).mean(axis=0)

array([0.56941923, 0.50666932])

In [20]:
np.stack(best_bilstm_history_20["recall_scores"]).mean(axis=0)

array([0.43776563, 0.634     ])

## Unidirectional

In [21]:
size = 20
bilstm_history_20_uni, best_bilstm_history_20_uni, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=False,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_uni.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_20_focal_2_uni_best_model.csv


In [22]:
bilstm_history_20_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/3077874114.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.375923  0.529393  0.525727   
                        0.0005         0.574045  0.504925  0.503486   
                        0.0010         0.500438  0.517318  0.515113   
           0.5          0.0001         0.434192  0.526851  0.525939   
                        0.0005         0.334952  0.545917  0.545779   
                        0.0010         0.482948  0.508103  0.505418   
200        0.1          0.0001         0.604255  0.511281  0.507539   
                        0.0005         0.526216  0.496981  0.493830   
                        0.0010         0.359506  0.519860  0.513294   
           0.5          0.0001         0.386797  0.540833  0.539747   
                        0.0005         0.450616  0.513505  0.510669   
                        0.0010         0.324613  0.545599  0.544072   
300        0.1          0.0001         0.330148  0.543375  0.535442   
                        0.0005         0.400610  0.541150  0.540126   
                        0.0010         0.431124  0.521131  0.513886   
           0.5          0.0001         0.305240  0.551954  0.543434   
                        0.0005         0.436730  0.530029  0.529088   
                        0.0010         0.500096  0.506514  0.499017   
384        0.1          0.0001         0.361563  0.536702  0.534692   
                        0.0005         0.426149  0.519860  0.504633   
                        0.0010         0.443130  0.531300  0.528622   
           0.5          0.0001         0.513900  0.534160  0.529980   
                        0.0005         0.417750  0.524627  0.522198   
                        0.0010         0.336386  0.554179  0.552261   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.531983  0.530551    0.325903   
                        0.0005          0.507260  0.506401    0.469888   
                        0.0010          0.517462  0.517438    0.424599   
           0.5          0.0001          0.526121  0.526218    0.368458   
                        0.0005          0.546565  0.546605    0.298152   
                        0.0010          0.506868  0.506968    0.389846   
200        0.1          0.0001          0.517084  0.516311    0.438023   
                        0.0005          0.496945  0.496997    0.403417   
                        0.0010          0.518634  0.519301    0.300434   
           0.5          0.0001          0.544741  0.543979    0.324551   
                        0.0005          0.512040  0.512486    0.370226   
                        0.0010          0.546467  0.545974    0.290421   
300        0.1          0.0001          0.557076  0.549650    0.282833   
                        0.0005          0.541112  0.541010    0.346498   
                        0.0010          0.519963  0.519147    0.344595   
           0.5          0.0001          0.567072  0.558204    0.274864   
                        0.0005          0.532135  0.531753    0.393167   
                        0.0010          0.504259  0.504081    0.431430   
384        0.1          0.0001          0.540722  0.539675    0.330117   
                        0.0005          0.514071  0.514362    0.367366   
                        0.0010          0.530300  0.529992    0.387180   
           0.5          0.0001          0.541511  0.539210    0.447792   
                        0.0005          0.524793  0.524509    0.378593   
                        0.0010          0.553858  0.553308    0.303152   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.715303  0.662367   
                        0.0005               0.734282  0.681575   
   

In [23]:
best_bilstm_history_20_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.556056,0.517636,0.512174,"[0.5637931034482758, 0.4605543710021322]",0.514169,"[0.5351882160392799, 0.4931506849315068]",0.513814,"[0.5956284153005464, 0.432]",0.507605,0.750890,...,False,384,0.5,0.0005,1,focal,2,False,None,64
0,0.305970,0.534795,0.534388,"[0.5481481481481482, 0.5206286836935167]",0.534511,"[0.5574387947269304, 0.5115830115830116]",0.534581,"[0.5391621129326047, 0.53]",0.299088,0.736655,...,False,384,0.5,0.0005,12,focal,2,False,None,64
0,0.391223,0.521449,0.520032,"[0.4939516129032258, 0.5461121157323688]",0.525699,"[0.5530474040632054, 0.49834983498349833]",0.525133,"[0.44626593806921677, 0.604]",0.329086,0.736655,...,False,384,0.5,0.0005,123,focal,2,False,None,64


In [24]:
best_bilstm_history_20_uni["f1"].mean()

0.522198005821278

In [25]:
best_bilstm_history_20_uni["precision"].mean()

0.524792991054572

In [26]:
best_bilstm_history_20_uni["recall"].mean()

0.5245094110503946

In [27]:
np.stack(best_bilstm_history_20_uni["f1_scores"]).mean(axis=0)

array([0.53529762, 0.50909839])

In [28]:
np.stack(best_bilstm_history_20_uni["precision_scores"]).mean(axis=0)

array([0.54855814, 0.50102784])

In [29]:
np.stack(best_bilstm_history_20_uni["recall_scores"]).mean(axis=0)

array([0.52701882, 0.522     ])

## history_length = 35

In [30]:
size = 50
bilstm_history_35, best_bilstm_history_35, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2_best_model.csv


In [31]:
bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/1307650554.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_35.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.283403  0.557674  0.554032   
                        0.0005         0.367627  0.554179  0.549443   
                        0.0010         0.495221  0.550365  0.538787   
           0.5          0.0001         0.293705  0.561487  0.560097   
                        0.0005         0.442160  0.537973  0.529940   
                        0.0010         0.451812  0.554496  0.550360   
200        0.1          0.0001         0.375932  0.526851  0.522914   
                        0.0005         0.334383  0.557674  0.544075   
                        0.0010         0.429819  0.546552  0.536774   
           0.5          0.0001         0.373348  0.527804  0.517106   
                        0.0005         0.315362  0.551319  0.533484   
                        0.0010         0.530575  0.563076  0.550649   
300        0.1          0.0001         0.325034  0.533842  0.529630   
                        0.0005         0.490711  0.543375  0.538921   
                        0.0010         0.393461  0.549412  0.537700   
           0.5          0.0001         0.343224  0.545599  0.534510   
                        0.0005         0.394083  0.545917  0.539306   
                        0.0010         0.410226  0.557674  0.538370   
384        0.1          0.0001         0.342562  0.521131  0.513536   
                        0.0005         0.404562  0.548141  0.533218   
                        0.0010         0.371629  0.548459  0.541875   
           0.5          0.0001         0.320414  0.522085  0.517378   
                        0.0005         0.407436  0.535113  0.527917   
                        0.0010         0.630232  0.562758  0.543130   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.565003  0.562121    0.252690   
                        0.0005          0.558247  0.555658    0.256364   
                        0.0010          0.564022  0.554931    0.308068   
           0.5          0.0001          0.566666  0.565199    0.265508   
                        0.0005          0.540502  0.538033    0.291822   
                        0.0010          0.556305  0.554504    0.259507   
200        0.1          0.0001          0.530473  0.528955    0.291114   
                        0.0005          0.575935  0.562419    0.239815   
                        0.0010          0.563254  0.552121    0.265509   
           0.5          0.0001          0.528881  0.527872    0.312871   
                        0.0005          0.573329  0.558192    0.238888   
                        0.0010          0.583503  0.569395    0.272866   
300        0.1          0.0001          0.539630  0.537687    0.257482   
                        0.0005          0.547048  0.544831    0.309308   
                        0.0010          0.568295  0.557858    0.244461   
           0.5          0.0001          0.560554  0.551954    0.268551   
                        0.0005          0.556513  0.551871    0.273163   
                        0.0010          0.587443  0.568161    0.254639   
384        0.1          0.0001          0.528921  0.526049    0.273150   
                        0.0005          0.571608  0.555245    0.258617   
                        0.0010          0.559849  0.555073    0.238888   
           0.5          0.0001          0.527010  0.525443    0.263784   
                        0.0005          0.537412  0.535093    0.247659   
                        0.0010          0.594541  0.573167    0.317104   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.709371  0.664723   
                        0.0005               0.727165  0.686887   
   

In [32]:
best_bilstm_history_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,1.194463,0.566254,0.536447,"[0.41890166028097064, 0.6539923954372623]",0.614231,"[0.7008547008547008, 0.5276073619631901]",0.579362,"[0.2987249544626594, 0.86]",0.470731,0.725979,...,True,384,0.5,0.001,1,focal,2,False,None,64
0,0.326094,0.561487,0.537478,"[0.4320987654320988, 0.6428571428571428]",0.597939,"[0.6704980842911877, 0.5253807106598984]",0.573381,"[0.31876138433515483, 0.828]",0.248915,0.718861,...,True,384,0.5,0.001,12,focal,2,False,None,64
0,0.370141,0.560534,0.555466,"[0.5080042689434365, 0.602928509905254]",0.571451,"[0.6134020618556701, 0.529500756429652]",0.566758,"[0.4335154826958106, 0.7]",0.231668,0.701068,...,True,384,0.5,0.001,123,focal,2,False,None,64


In [33]:
best_bilstm_history_35["f1"].mean()

0.5431304571426941

In [34]:
best_bilstm_history_35["precision"].mean()

0.5945406126757166

In [35]:
best_bilstm_history_35["recall"].mean()

0.5731669702489374

In [36]:
np.stack(best_bilstm_history_35["f1_scores"]).mean(axis=0)

array([0.45300156, 0.63325935])

In [37]:
np.stack(best_bilstm_history_35["precision_scores"]).mean(axis=0)

array([0.66158495, 0.52749628])

In [38]:
np.stack(best_bilstm_history_35["recall_scores"]).mean(axis=0)

array([0.35033394, 0.796     ])

## Unidirectional

In [39]:
size = 50
bilstm_history_35_uni, best_bilstm_history_35_uni, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=False,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_uni.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_35_focal_2_uni_best_model.csv


In [40]:
bilstm_history_35_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/2665543288.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_35_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.323338  0.535748  0.529944   
                        0.0005         0.397495  0.552908  0.549027   
                        0.0010         0.479235  0.550365  0.546080   
           0.5          0.0001         0.352080  0.571973  0.571238   
                        0.0005         0.329795  0.566889  0.564681   
                        0.0010         0.379032  0.536384  0.535686   
200        0.1          0.0001         0.382449  0.555132  0.554148   
                        0.0005         0.454626  0.560216  0.555600   
                        0.0010         0.328483  0.545917  0.533762   
           0.5          0.0001         0.443744  0.546235  0.545068   
                        0.0005         0.437705  0.552590  0.546564   
                        0.0010         0.320256  0.570067  0.566901   
300        0.1          0.0001         0.343343  0.534160  0.522179   
                        0.0005         0.332125  0.556403  0.549826   
                        0.0010         0.368779  0.558309  0.556849   
           0.5          0.0001         0.382263  0.555767  0.546071   
                        0.0005         0.405762  0.542739  0.535728   
                        0.0010         0.471984  0.538608  0.534781   
384        0.1          0.0001         0.442155  0.542739  0.542430   
                        0.0005         0.305204  0.557992  0.550334   
                        0.0010         0.393990  0.553225  0.546053   
           0.5          0.0001         0.342623  0.548141  0.538959   
                        0.0005         0.500512  0.540515  0.538324   
                        0.0010         0.406695  0.546870  0.543106   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.537406  0.535641    0.287399   
                        0.0005          0.560888  0.556883    0.321444   
                        0.0010          0.550505  0.549159    0.416611   
           0.5          0.0001          0.575138  0.574355    0.300836   
                        0.0005          0.572451  0.570211    0.287456   
                        0.0010          0.537952  0.537527    0.339646   
200        0.1          0.0001          0.559312  0.558294    0.329862   
                        0.0005          0.566639  0.563033    0.379929   
                        0.0010          0.549890  0.546843    0.286131   
           0.5          0.0001          0.548956  0.548366    0.363810   
                        0.0005          0.563854  0.557800    0.360863   
                        0.0010          0.572813  0.570927    0.291573   
300        0.1          0.0001          0.544566  0.537336    0.284564   
                        0.0005          0.569406  0.561234    0.287762   
                        0.0010          0.557914  0.557552    0.327011   
           0.5          0.0001          0.570404  0.561014    0.296855   
                        0.0005          0.547820  0.544491    0.335907   
                        0.0010          0.542949  0.541229    0.408168   
384        0.1          0.0001          0.544084  0.543896    0.383593   
                        0.0005          0.572275  0.563199    0.262564   
                        0.0010          0.561201  0.556086    0.321767   
           0.5          0.0001          0.562389  0.554918    0.296145   
                        0.0005          0.543636  0.542277    0.463164   
                        0.0010          0.548592  0.546920    0.364886   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.720047  0.667392   
                        0.0005               0.733096  0.680516   
   

In [41]:
best_bilstm_history_35_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.364149,0.542421,0.538952,"[0.4989561586638831, 0.5789473684210525]",0.549989,"[0.5843520782396088, 0.515625]",0.547668,"[0.4353369763205829, 0.66]",0.329813,0.743772,...,False,384,0.5,0.0001,1,focal,2,False,None,64
0,0.374256,0.551954,0.550808,"[0.5281124497991968, 0.573502722323049]",0.556642,"[0.5883668903803132, 0.5249169435215947]",0.555526,"[0.4790528233151184, 0.632]",0.327832,0.747331,...,False,384,0.5,0.0001,12,focal,2,False,None,64
0,0.289464,0.550048,0.527116,"[0.4229828850855745, 0.63125]",0.580536,"[0.6431226765799256, 0.517948717948718]",0.561559,"[0.3151183970856102, 0.808]",0.230791,0.708185,...,False,384,0.5,0.0001,123,focal,2,False,None,64


In [42]:
best_bilstm_history_35_uni["f1"].mean()

0.538958597382126

In [43]:
best_bilstm_history_35_uni["precision"].mean()

0.5623887177783601

In [44]:
best_bilstm_history_35_uni["recall"].mean()

0.5549180327868853

In [45]:
np.stack(best_bilstm_history_35_uni["f1_scores"]).mean(axis=0)

array([0.4833505, 0.5945667])

In [46]:
np.stack(best_bilstm_history_35_uni["precision_scores"]).mean(axis=0)

array([0.60528055, 0.51949689])

In [47]:
np.stack(best_bilstm_history_35_uni["recall_scores"]).mean(axis=0)

array([0.40983607, 0.7       ])

## history_length = 50

In [50]:
size = 50
bilstm_history_50, best_bilstm_history_50, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_50_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_50_focal_2_best_model.csv


In [51]:
bilstm_history_50.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/1298888840.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_50.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.308573  0.559898  0.556181   
                        0.0005         0.367520  0.556721  0.547914   
                        0.0010         0.498473  0.562123  0.543566   
           0.5          0.0001         0.303906  0.577375  0.572376   
                        0.0005         0.357183  0.557038  0.546301   
                        0.0010         0.340354  0.557356  0.543603   
200        0.1          0.0001         0.347706  0.550048  0.544047   
                        0.0005         0.369215  0.544010  0.535336   
                        0.0010         0.361638  0.561169  0.545066   
           0.5          0.0001         0.420866  0.555767  0.551771   
                        0.0005         0.472432  0.541468  0.531985   
                        0.0010         0.367408  0.551636  0.536320   
300        0.1          0.0001         0.356119  0.537655  0.534215   
                        0.0005         0.319248  0.546552  0.532228   
                        0.0010         0.373755  0.555132  0.539144   
           0.5          0.0001         0.485682  0.537973  0.534660   
                        0.0005         0.395522  0.544963  0.541729   
                        0.0010         0.422622  0.561169  0.546973   
384        0.1          0.0001         0.332671  0.541468  0.539019   
                        0.0005         0.330896  0.550683  0.533631   
                        0.0010         0.356929  0.542421  0.528165   
           0.5          0.0001         0.342435  0.552908  0.550806   
                        0.0005         0.339491  0.552590  0.531049   
                        0.0010         0.684751  0.547823  0.531661   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.567850  0.564425    0.244076   
                        0.0005          0.570537  0.561359    0.274750   
                        0.0010          0.586810  0.569257    0.311242   
           0.5          0.0001          0.582972  0.578266    0.241361   
                        0.0005          0.571602  0.560979    0.269153   
                        0.0010          0.583562  0.565745    0.248501   
200        0.1          0.0001          0.561264  0.556621    0.251506   
                        0.0005          0.556376  0.550674    0.271088   
                        0.0010          0.589562  0.570310    0.242900   
           0.5          0.0001          0.564744  0.561252    0.293142   
                        0.0005          0.554814  0.548513    0.302845   
                        0.0010          0.576017  0.560757    0.266306   
300        0.1          0.0001          0.544668  0.542787    0.270478   
                        0.0005          0.566409  0.555155    0.237337   
                        0.0010          0.580250  0.564810    0.239150   
           0.5          0.0001          0.544550  0.542704    0.295079   
                        0.0005          0.548431  0.546854    0.271936   
                        0.0010          0.584800  0.570400    0.246695   
384        0.1          0.0001          0.546588  0.545181    0.277538   
                        0.0005          0.579250  0.559132    0.238259   
                        0.0010          0.564291  0.551149    0.235472   
           0.5          0.0001          0.557239  0.555902    0.266642   
                        0.0005          0.586435  0.563304    0.236862   
                        0.0010          0.576870  0.556965    0.338713   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.714116  0.670134   
                        0.0005               0.721234  0.685717   
   

In [52]:
best_bilstm_history_50

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.522368,0.561487,0.531110,"[0.411764705882353, 0.6504559270516718]",0.607748,"[0.6909871244635193, 0.5245098039215687]",0.574630,"[0.29326047358834245, 0.856]",0.299781,0.715302,...,True,384,0.1,0.001,1,focal,2,False,None,64
0,0.276995,0.534795,0.528310,"[0.4730021598272138, 0.5836177474402731]",0.544915,"[0.5809018567639257, 0.5089285714285714]",0.541454,"[0.3989071038251366, 0.684]",0.199421,0.715302,...,True,384,0.1,0.001,12,focal,2,False,None,64
0,0.271424,0.530982,0.525074,"[0.4721030042918455, 0.5780445969125214]",0.540209,"[0.5744125326370757, 0.506006006006006]",0.537364,"[0.4007285974499089, 0.674]",0.207214,0.729537,...,True,384,0.1,0.001,123,focal,2,False,None,64


In [53]:
best_bilstm_history_50["f1"].mean()

0.5281646902343131

In [54]:
best_bilstm_history_50["precision"].mean()

0.5642909825367778

In [55]:
best_bilstm_history_50["recall"].mean()

0.5511493624772313

In [56]:
np.stack(best_bilstm_history_50["f1_scores"]).mean(axis=0)

array([0.45228996, 0.60403942])

In [57]:
np.stack(best_bilstm_history_50["precision_scores"]).mean(axis=0)

array([0.61543384, 0.51314813])

In [58]:
np.stack(best_bilstm_history_50["recall_scores"]).mean(axis=0)

array([0.36429872, 0.738     ])

## Unidirectional

In [59]:
size = 50
bilstm_history_50_uni, best_bilstm_history_50_uni, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=False,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_uni.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_50_focal_2_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/lstm_history_50_focal_2_uni_best_model.csv


In [60]:
bilstm_history_50_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_537867/972659332.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_50_uni.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
100        0.1          0.0001         0.321718  0.567525  0.564948   
                        0.0005         0.366988  0.548459  0.544661   
                        0.0010         0.528008  0.526533  0.514880   
           0.5          0.0001         0.358694  0.580235  0.579976   
                        0.0005         0.356399  0.568796  0.565183   
                        0.0010         0.429581  0.554496  0.550842   
200        0.1          0.0001         0.399494  0.563711  0.558741   
                        0.0005         0.329617  0.558627  0.549408   
                        0.0010         0.345803  0.563076  0.556696   
           0.5          0.0001         0.376615  0.558627  0.554925   
                        0.0005         0.381754  0.561169  0.549166   
                        0.0010         0.394962  0.571656  0.567392   
300        0.1          0.0001         0.321053  0.564665  0.556656   
                        0.0005         0.379950  0.550683  0.545577   
                        0.0010         0.334709  0.567207  0.549596   
           0.5          0.0001         0.279676  0.555450  0.543170   
                        0.0005         0.417495  0.559898  0.550944   
                        0.0010         0.372336  0.546234  0.541202   
384        0.1          0.0001         0.379497  0.557992  0.549404   
                        0.0005         0.378998  0.547188  0.544902   
                        0.0010         0.356240  0.525898  0.521051   
           0.5          0.0001         0.330855  0.559263  0.553349   
                        0.0005         0.388178  0.544963  0.544532   
                        0.0010         0.369844  0.565300  0.552435   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
100        0.1          0.0001          0.572111  0.570134    0.287906   
                        0.0005          0.556066  0.552217    0.286125   
                        0.0010          0.521036  0.521362    0.413482   
           0.5          0.0001          0.583017  0.582426    0.310080   
                        0.0005          0.577669  0.573610    0.285974   
                        0.0010          0.552850  0.551945    0.423116   
200        0.1          0.0001          0.574671  0.569466    0.305385   
                        0.0005          0.578431  0.564877    0.288045   
                        0.0010          0.571955  0.566271    0.277891   
           0.5          0.0001          0.566785  0.563330    0.286739   
                        0.0005          0.579990  0.566026    0.301902   
                        0.0010          0.579654  0.575955    0.312798   
300        0.1          0.0001          0.579421  0.571180    0.300432   
                        0.0005          0.555858  0.552795    0.304151   
                        0.0010          0.598150  0.577536    0.250891   
           0.5          0.0001          0.574698  0.563239    0.254523   
                        0.0005          0.576820  0.567281    0.317409   
                        0.0010          0.553460  0.549526    0.315050   
384        0.1          0.0001          0.573757  0.565489    0.327968   
                        0.0005          0.553130  0.551419    0.323465   
                        0.0010          0.532337  0.529264    0.273829   
           0.5          0.0001          0.571151  0.565692    0.285431   
                        0.0005          0.547606  0.547152    0.316571   
                        0.0010          0.588828  0.574108    0.283538   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
100        0.1          0.0001               0.720047  0.668152   
                        0.0005               0.735469  0.682609   
   

In [61]:
best_bilstm_history_50_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.327470,0.546234,0.545785,"[0.531496062992126, 0.5600739371534197]",0.549389,"[0.5781584582441114, 0.520618556701031]",0.548902,"[0.4918032786885246, 0.606]",0.321424,0.743772,...,False,384,0.1,0.0005,1,focal,2,False,None,64
0,0.335829,0.552908,0.551131,"[0.5228891149542217, 0.579372197309417]",0.558685,"[0.5921658986175116, 0.5252032520325203]",0.557062,"[0.4681238615664845, 0.646]",0.283601,0.740214,...,False,384,0.1,0.0005,12,focal,2,False,None,64
0,0.473695,0.542421,0.537790,"[0.49152542372881364, 0.5840554592720971]",0.551316,"[0.5873417721518988, 0.5152905198776758]",0.548293,"[0.4225865209471767, 0.674]",0.365368,0.743772,...,False,384,0.1,0.0005,123,focal,2,False,None,64


In [62]:
best_bilstm_history_50_uni["f1"].mean()

0.5449020325683492

In [63]:
best_bilstm_history_50_uni["precision"].mean()

0.5531297429374581

In [64]:
best_bilstm_history_50_uni["recall"].mean()

0.5514189435336977

In [65]:
np.stack(best_bilstm_history_50_uni["f1_scores"]).mean(axis=0)

array([0.51530353, 0.57450053])

In [66]:
np.stack(best_bilstm_history_50_uni["precision_scores"]).mean(axis=0)

array([0.58588871, 0.52037078])

In [67]:
np.stack(best_bilstm_history_50_uni["recall_scores"]).mean(axis=0)

array([0.46083789, 0.642     ])